# 

# Here onwards i am build a chat appliation using RAG Framework

* Install the required packages

In [1]:
%pip install --upgrade --quiet langchain
%pip install --upgrade --quiet pinecone
%pip install --upgrade --quiet langchain-google-genai pillow
%pip install --upgrade --quiet langchain-pinecone
%pip install --upgrade --quiet python-dotenv pypdf

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


* Global imports

In [3]:
import os
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone as PineconeClient
from dotenv import load_dotenv

* env-variables loading

In [4]:
load_dotenv()
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_API_HOST = os.getenv("PINECONE_API_HOST")
PINECONE_API_INDEX = os.getenv("PINECONE_API_INDEX")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

* Load the data/Extract the data from .pdf

In [5]:
def load_pdf(data):
    loader = DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyPDFLoader,
    )
    documents = loader.load()
    return documents

In [6]:
extracted_data = load_pdf("../data/")

In [ ]:
# extracted_data

* Data -> text_chunks

In [7]:
def text_split(extracted_data):
    text_spliter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20
    )
    text_chunks = text_spliter.split_documents(extracted_data)
    return text_chunks

In [8]:
text_chunks = text_split(extracted_data)

In [9]:
print(f'length of total chunks is {len(text_chunks)}')

length of total chunks is 7020


* Download the embedding model from langchain_google_genai

In [10]:
def download_hugging_face_embedding():
    embedding = GoogleGenerativeAIEmbeddings(
        model="models/text-embedding-004",
        google_api_key=GOOGLE_API_KEY
    )
    return embedding

In [11]:
embedding = download_hugging_face_embedding()

In [12]:
embedding

GoogleGenerativeAIEmbeddings(client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x10e840860>, model='models/text-embedding-004', task_type=None, google_api_key=SecretStr('**********'), credentials=None, client_options=None, transport=None, request_options=None)

* Pinecode initializing

In [13]:
pc = PineconeClient(api_key=PINECONE_API_KEY)

In [14]:
index_name = PINECONE_API_INDEX

In [ ]:
# dosearch = PineconeVectorStore.from_texts(
#     [
#         tc.page_content for tc in text_chunks
#     ],
#     embedding=embedding,
#     index_name=index_name,
# )

In [15]:
docsearch = PineconeVectorStore.from_existing_index(index_name=index_name, embedding=embedding)

In [16]:
docsearch

* testing

In [ ]:
query = "what is acne?"
docs = docsearch.similarity_search(query, k=4)
print(docs)

* Prompt Templating

In [ ]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [ ]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

* Llm setup

In [ ]:
def get_llm():
    llm = ChatGoogleGenerativeAI(
        model="gemini-pro",
        google_api_key=GOOGLE_API_KEY,
        temperature=1,
        max_output_tokens=512,
        min_output_tokens=12,
    )
    return llm


In [ ]:
llm = get_llm()

In [ ]:
llm

ChatGoogleGenerativeAI(model='models/gemini-pro', google_api_key=SecretStr('**********'), temperature=1.0, max_output_tokens=512, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x1765199d0>, async_client=<google.ai.generativelanguage_v1beta.services.generative_service.async_client.GenerativeServiceAsyncClient object at 0x1765abf20>, default_metadata=())

* Llm chains

In [ ]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever(search_kwargs={'k': 7}),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)

In [ ]:
res = qa({"query": "what are acne"})

In [ ]:
print(res)

In [ ]:
print(res["result"])

In [ ]:
"done🤘🏻"

'done🤘🏻'